In [128]:
path ='/Users/taehunlee/Desktop/duty_schedule_ver2/test1.xlsx'

In [81]:
# from google.colab import drive
# drive.mount('/content/drive')

# 1. 필요한 라이브러리 인스톨 및 로딩(Launching and Loading libraries)

In [82]:
# !pip install holidays-jp
#There is API for JP Holidays to get information from google calendar. https://holidays-jp.github.io/

#If I USE Google calenader api, Maybe I will not need to update holidays of Japan
#https://pypi.org/project/japan_holiday/ It will be helpful to me


In [129]:
import requests
import json
from datetime import datetime
from collections import OrderedDict
import pandas as pd
import numpy as np
import random
import calendar
# from holidays_jp import CountryHolidays
from collections import Counter
import re

# Test and Practice of function


## 1. Function Check if All of members can handle with holidays duty of this month ,Defining old boys have only once of holiday duty

>1.1 Equation of check availability of This month

>> 1.1.1 x: number of no no_duty members

>> 1.1.2 y: number of old boys

>> 1.1.3 z: total number of holiday duty of ICU and CALL

>> 1.1.4 Caution!

>>>1.1.4.1 There are specific members only having duty days assigned by themselves.

>>>>上田）only have a ICU duty assigned by himself and Call duty is only on weekday.

>>>>猪原）She has no additional duty except to days assigned by herself

>>>1.1.4.2 If there are enough availability, Old boy should be charged of 2 days of holiday duty. the method is defining order like (荒井＞清水と＞井上＞北) or assigning by Master(北)




>>1.1.5 So, Availability is same or more than 4. because there is possibility that 猪原 doesn't assign holiday duty by herself.

>>> $$(2x-y)-z > 3 $$

Old boys are ['北', '清水と', '荒井', '柴田']
週末の当直ができる人員数：18
Old boyで週末当直1回にする人員数：4
今月に週末当直の総数：28
holiday_availability is 4
Required number of twice old boy is 2
Members of old boys take twice of duty on holiday ['柴田', '荒井']


In [126]:
a

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李,no_holidays
no_icu_duty,True,True,True,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,NaN
no_cardiac_surgery,True,False,False,False,True,True,False,False,False,True,False,True,False,False,False,False,False,False,NaN
no_additional_duty,True,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,NaN
old_boy,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN
unavailable_days,"[6, 7, 20, 21, 22, 23]","[5, 22, 23, 28]","[1, 2, 5, 15, 20, 21, 22]","[5, 6]","[1, 2, 5, 7, 14, 15, 16, 22, 28]","[1, 6, 7, 8, 12, 13, 14, 15, 16, 21, 28]","[1, 5, 7, 8, 9, 10, 12, 14, 15, 16, 22, 23, 26...","[6, 16]",[6],"[5, 7, 9, 21, 27]",[5],"[1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 17, 18, 24, 2...","[1, 2, 6, 8, 9, 10, 15, 16, 29]","[5, 7, 14, 15, 16, 29]","[1, 2, 6, 14, 15, 21, 22]","[1, 2, 3, 5, 7, 10, 17, 21, 22, 23, 24, 28]","[5, 7]","[5, 6, 7, 8, 20, 21, 22]",NaN
hope_icu_duty,[],[],[],[],[],"[3, 17]",[],[],[],"[1, 16, 22, 30]",[],[],[],[],"[19, 7]","[26, 4, 11]",[],[],NaN
hope_call_duty,[8],[],[],[],[29],[],"[2, 21, 25]",[],[],[],[],[],[],[],[],[],[],[],NaN
icu_duty,[],[],[],"[15, 24]",[],[],[],"[8, 12]","[21, 25]",[],"[14, 20]","[28, 5]","[23, 18]","[9, 13]",[],[],"[29, 10, 6]","[2, 27]",NaN
call_duty,[],"[1, 18, 4]","[23, 27, 17]","[11, 19]",[],[],[],"[28, 24, 5]",[9],[],"[7, 3]","[14, 20]","[30, 12]","[22, 6]","[16, 13]",[],[26],"[15, 10]",NaN
duty_score,-100,5,5,11,-100,-100,5,11,11,-100,11,11,11,11,11,11,11,11,NaN


# 2. 여러번 반복사용하는 구문들은 함수로 정의(Functions for repeated code)

  -Get weekend of defined month
  -Get holiday of japan
  -remove_list(list1, list2)
  -check_list(list1, list2)
  -members_info_refresh(members,no_icu_duty,no_cardiac_surgery,unavavailable_days  )

##2.1 Functions for making information table of members

In [131]:
#Make a dict including information of all members from dataframe

def make_members_info(membs_info, mem_info):
    members_info = {}

    #Take a information about skill or limitation of all members from dataframe
    no_cardiac_surgery = list(mem_info.columns[(mem_info[mem_info['name'] == 'no_cardiac_surgery'] == 1).iloc[0]])
    no_icu_duty = list(mem_info.columns[(mem_info[mem_info['name'] == 'no_icu_duty'] == 1).iloc[0]])
    no_additional_duty = list(mem_info.columns[(mem_info[mem_info['name'] == 'no_additional_duty'] == 1).iloc[0]])
    old_boy = list(mem_info.columns[(mem_info[mem_info['name'] == 'old_boy'] == 1).iloc[0]])
    #Check if information of members about skill and limitation is correct
    print("心臓手術非対応　: ",no_cardiac_surgery)
    print("ICU当直非対応 : ",no_icu_duty)
    print("指定Duty以外は追加無し : ",no_additional_duty)
    print("OLD BOY : ",old_boy)

    #Make a dict item. Key is a name of each member and Values are any information of skill, limitation, hoped_duty and duty_day
    for name in membs_info.columns:

        #make a base frame of dict
        members_info[name] = {
            'no_icu_duty' : False,
            'no_cardiac_surgery' :False,
            'no_additional_duty' : False,
            "old_boy" : False,
            'unavailable_days': [],
            'hope_icu_duty': [],
            'hope_call_duty': [],
            'icu_duty' :[],
            'call_duty' :[],
            'duty_score' :11,
            'no_holidays' :0,
            'duty_num': 0,
            }

        #Add information about duty days. the data is datetime type
        members_info[name]['unavailable_days'].extend(membs_info[membs_info[name] == 1].index.tolist())
        members_info[name]['hope_icu_duty'].extend(membs_info[membs_info[name] == 2].index.tolist())
        members_info[name]['hope_call_duty'].extend(membs_info[membs_info[name] == 3].index.tolist())

        #Insert a duty_num inputed for each member by Master
        members_info[name]['duty_num']= int(mem_info.loc[mem_info['name']=='duty_num',name])

        #Extract only the day from datetime to use only number.
        members_info[name]['unavailable_days']= [n.day for n in members_info[name]['unavailable_days']]
        members_info[name]['hope_icu_duty']= [n.day for n in members_info[name]['hope_icu_duty']]
        members_info[name]['hope_call_duty']= [n.day for n in members_info[name]['hope_call_duty']]

        #Adjust basic information based on characteristics of Each Member
        if name in no_icu_duty: #Set duty_ score of the members in no_icu_duty list. unless set the score like this the members get so many call duty.
            members_info[name]['no_icu_duty'] = True
            members_info[name]['duty_score'] = 5 # This point is set under of base score.
        if name in no_cardiac_surgery: #
            members_info[name]['no_cardiac_surgery'] = True
        if name in no_additional_duty: #Set the duty_score of members in no_additional_duty list to exclude additional selection from the duty schedule
            members_info[name]['no_additional_duty'] = True
            members_info[name]['duty_score'] = -100
        if name in old_boy: #Set the no_holidays as 1 because in this algorithm old boy will get 2 duty on holidays.
            members_info[name]['old_boy'] = True
            members_info[name]['no_holidays'] = 1 #But if there are many days of holidays, old boy also get a duty work on 2 holidays.

    return members_info


In [132]:
#Take information from dataframe containing characteristics of each member
def list_memb_skill(df_mem_info):
    no_cardiac_surgery = list(df_mem_info.columns[(df_mem_info[df_mem_info['name'] == 'no_cardiac_surgery'] == 1).iloc[0]])
    no_icu_duty = list(df_mem_info.columns[(df_mem_info[df_mem_info['name'] == 'no_icu_duty'] == 1).iloc[0]])
    no_additional_duty = list(df_mem_info.columns[(df_mem_info[df_mem_info['name'] == 'no_additional_duty'] == 1).iloc[0]])
    old_boy = list(df_mem_info.columns[(df_mem_info[df_mem_info['name'] == 'old_boy'] == 1).iloc[0]])
    return no_cardiac_surgery, no_icu_duty, no_additional_duty , old_boy

In [133]:
#Check available number of memebrs on holiday duty. And, if the members is deficient, Some members of old boys take a duty on holidays twice.
#This functions require 2 dataframes and 2 data of classified day and old boys.
#Compnay's availability for holiday duty is 32 times(duty member *2 + old boy *1) per month. So, Considering events that make some member unavailable of holiday duty, There are 4 times as a 

#Available total count of holiday duty days of members
    #1. Old boy has 1 count
    #2. No old boy has 2 count
        #2.1 No old boy except no_duty members has 2 count
        #2.2 So, total count is twice of members excluding old boys and no_duty members
    #3. But if Available total count is less than total count of holiday duty days, Old boy is charged of 2 counts.

def select_twice_old_boy(df_mem_info = df_mem_info, 
                         info_days = info_days, 
                         members_df = members_df,
                         old_boy = old_boy):
    print(f"Old boys are {old_boy}")
    #count of no no_duty members as x
    total_memb = df_mem_info.columns[1:]
    total_num_memb = len(total_memb)

    #members able to do duty as x
    row_no_duty = df_mem_info.loc[df_mem_info["name"]=="no_duty"]
    num_df_memb_no_duty = row_no_duty.select_dtypes(include=["int"])
    no_duty_num = num_df_memb_no_duty.iloc[0].sum()
    total_able_number = total_num_memb - no_duty_num
    print(f"週末の当直ができる人員数：{total_able_number}")
    #members of old boy as y
    row_no_duty = df_mem_info.loc[df_mem_info["name"]=="old_boy"]
    num_df_memb_old_boy = row_no_duty.select_dtypes(include=["int"])
    old_boy_num = num_df_memb_old_boy.iloc[0].sum()
    print(f"Old boyで週末当直1回にする人員数：{old_boy_num}")

    #total count of duty days of holiday as z
    total_holi_duty = len(sorted(info_days[0] + info_days[2]+info_days[1]))*2
    print(f"今月に週末当直の総数：{total_holi_duty}")

    #Check eqauation : (2x-y)-z > 3
    holiday_availability = total_able_number*2-old_boy_num-total_holi_duty
    print(f"holiday_availability is {holiday_availability}")

    holiday_availability = 0
    old_boy_order = old_boy[::-1]
    required_num = 0
    # if holiday_availability<4:
    required_num = (4-holiday_availability)//2+(4-holiday_availability)%2
    print(f"Required number of twice old boy is {required_num}")
    twice_old_boy = old_boy_order[:required_num]
    print(f"Members of old boys take twice of duty on holiday {twice_old_boy}")
    for selected_memb in twice_old_boy:
        members_df.loc["no_holidays",selected_memb] = 0

    return members_df   

## 2.2 Funtions to Classify days and get a date information

In [134]:
#Extract day from excel string of date like 2/14(金)
def extract_day(date_str):
    # 正規表現で括弧とその中身を削除
    date_str = re.sub(r'\([^)]*\)', '', date_str).strip()
    # 日付をパースして日を抽出
    month, day = date_str.split('/')
    return day

#def additiional_holidays is removed becaus it is not used.

#Get a extra holiday from excel file(Checked)
    #Maybe this function is useful to input additional company's holiday without manipulating get_holidays_from_api

def get_extra_holidays(df_extra_info):
    extra_days = []
    dt = df_extra_info["date"][df_extra_info["extra_holidays"]==1]
    for n in dt:
        print(n.day)
        extra_days.append(n.day)
    return extra_days


#Get a information of Japanese holiday through API based google calendar.
# Refrence : https://holidays-jp.github.io/ (MIT Licence)

def get_holidays_from_api(year):

    #Get a information about Japanese Holidays from Google calendar through by API
    api_url = f"https://holidays-jp.github.io/api/v1/{year}/date.json"
    response = requests.get(api_url)
    data = response.text
    parsed_data = json.loads(data)
    converted_data = {}
    for date_str, event in parsed_data.items():
        # Convert the date string to a datetime object
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        # Store the datetime object as the key in the new dictionary
        converted_data[date_obj] = event
    #Add holidays of Company
    lastyear_holidays = ((datetime(year,1,2,0,0), "年末年始休み"),
                        (datetime(year,1,3,0,0), "年末年始休み"),
                        (datetime(year,1,4,0,0), "年末年始休み"),
                        (datetime(year,12,30,0,0), "年末年始休み"),
                        (datetime(year,12,31,0,0), "年末年始休み"),)
    for holi_date, holi_name in lastyear_holidays:
        converted_data[holi_date] = holi_name

    sorted_converted_data = OrderedDict(sorted(converted_data.items(), key=lambda x: x[0]))
    print(f"Japanese Holidays of {year} : \n")
    return sorted_converted_data
#Holidays list from holidays-jp, Customized holidays of Company and Birthday of Jpanese King
# def holidays_of_this_year(year):

#     holi = CountryHolidays.get('JP', year)
#     if datetime(year,2,23,0,0).weekday() == 5:
#         holi.append((datetime(year,2,25,0,0),'振替休日'))
#     elif datetime(year,2,23,0,0).weekday() == 6:
#         holi.append((datetime(year,2,24,0,0),'振替休日'))
#     holi.append((datetime(year,2,23,0,0),'天皇誕生日'))

#     lastyear_holi = ((datetime(year,1,2,0,0), "年末年始休み"),
#                     (datetime(year,1,3,0,0), "年末年始休み"),
#                     (datetime(year,1,4,0,0), "年末年始休み"),
#                     (datetime(year,12,30,0,0), "年末年始休み"),
#                     (datetime(year,12,31,0,0), "年末年始休み"),)
#     for n in lastyear_holi:
#         holi.append(n)
#     sorted_holi = sorted(holi, key=lambda x: x[0])
#     return sorted_holi

# Check if first day of next month is holiday(Updated)

def check_first_day_of_next_month(year, month, holidays):
    next_month = month+1
    if month == 12:  #If this month is 12, make next_month 1
        next_month = 1
    holidays_date = list(holidays.keys())
    next_first = datetime(year, next_month, 1)
    if next_first in holidays_date:
        is_next_holiday = True
        print(f"the First day of Next Month is holiday")
        return is_next_holiday
    elif datetime(year,next_month,1,0,0).weekday() in [5,6]:
        is_next_holiday = True
        print(f"the First day of Next Month is holiday")
        return is_next_holiday
    else:
        is_next_holiday = False
        print(f"the First day of Next Month is NOT holiday")
        return is_next_holiday

#Divide days of selected month to weekdays and vacation_days including holidays
def distinct_days(year, month, extra_holiday, holidays):
    weekends = []
    day_holiday =[] #Removed useless variables#
    period = calendar.monthrange(year, month)[1]

    for day in range(1, period + 1):
        if calendar.weekday(year, month, day) in [5, 6]:  # 5=土曜日, 6=日曜日
            weekends.append(day)
    for date in holidays.keys():
            if date.month == month: #date is datetime type. So, date[0] is not needed
                day_holiday.append(date.day)
    else:
        # Assign a as a list containing None
        add_days = []
    if len(extra_holiday) != 0:
        print("There is extra holiday at this month.")
        integ_holiday = sorted(set(weekends + day_holiday + extra_holiday))
    if len(extra_holiday) == 0:
        print("There is NOT extra holiday at this month.")
        integ_holiday = sorted(set(weekends + day_holiday))
    print(f"Holidays of this month : {integ_holiday}")

    #Weekdays is remained days after Deleting holidays from of this month.
    weekdays = [day for day in range(1, period+1) if day not in integ_holiday]
    print(f"Weekdays of this month : {weekdays}")

    return weekdays, integ_holiday

#Classify weekday of a day before the holiday and the others(Updated)
def last_weekdays(weekdays, vacation_days, period, is_next_holiday):
    last_weekday = []
    no_last_weekdays = []
    for n in weekdays:
        if n+1 in vacation_days:
            last_weekday.append(n)
            print(f"{n} day is a last day of weekdays")

        elif weekdays[-1] == period\
            and is_next_holiday: #Remove unnecessary code
            last_weekday.append(n)
            print(f"{n} day is a last day of weekdays")

        else:
            print(f"{n} day is not a last day of weekdays")
            no_last_weekdays.append(n)

    return last_weekday, no_last_weekdays

#Determine if a day of vacation_days is last of long weekend.
def not_last_holiday(vacation_days, period, is_next_holiday):
    not_last_vacation_days = []
    last_vacation_day = []

    for n in vacation_days:
        if n+1 in vacation_days:
            not_last_vacation_days.append(n)
            print(f"{n} day is NOT a last day of vacation days")
            print(not_last_vacation_days)
        elif (n == vacation_days[-1] )\
            and vacation_days[-1] == period\
            and is_next_holiday:
            not_last_vacation_days.append(n)
            print(f"{n} day is NOT a last day of vacation_days")
            print(not_last_vacation_days)
        else:
            print(f"{n} day is a last day of vacation_days")
            last_vacation_day.append(n)

    return not_last_vacation_days, last_vacation_day

#Make a classified days by function and function(inform_days uses not_last_holiday and last_weekdays)
    #If the functions are sorted in order, a function can be nested within another function.

def inform_days(weekdays, vacation_days, period, is_next_holiday):
    not_last_vacation_days, last_vacation_days = not_last_holiday(vacation_days, period, is_next_holiday)
    last_weekday, not_last_weekdays = last_weekdays(weekdays, vacation_days, period, is_next_holiday)

    return not_last_vacation_days, last_vacation_days, last_weekday, not_last_weekdays


## 2.3 Functions to Decide members randomly

In [135]:
#Decide duty members
def hoped_duty(members_info, info_days, icu = True):
    if icu :
        info_days[0] = info_days[0]
        info_days[1] = info_days[1]
        info_days[2] = info_days[2]
        info_days[3] = info_days[3]

        for name in members_info.keys():
            if members_info[f'{name}']['hope_icu_duty'] :
                members_info[f'{name}']['icu_duty'] = members_info[f'{name}']['hope_icu_duty']
                for day in members_info[f'{name}']['hope_icu_duty']:
                    members_info[f'{name}']['duty_score'] = count_down_duty_score(name, day, members_info,
                                                                              info_days, icu)
                info_days[0] = remove_list(info_days[0], members_info[f'{name}']['icu_duty'])
                info_days[1] = remove_list(info_days[1], members_info[f'{name}']['icu_duty'])
                info_days[2] = remove_list(info_days[2], members_info[f'{name}']['icu_duty'])
                info_days[3] = remove_list(info_days[3], members_info[f'{name}']['icu_duty'])
    if icu == False:
        info_days[0] = info_days[0]
        info_days[1] = info_days[1]
        info_days[2] = info_days[2]
        info_days[3] = info_days[3]
        for name in members_info.keys():
            if members_info[f'{name}']['hope_call_duty'] :
                for day in members_info[f'{name}']['hope_call_duty']:
                    members_info[f'{name}']['duty_score'] = count_down_duty_score(name, day, members_info,
                                                                              info_days, icu)
                members_info[f'{name}']['call_duty'] = members_info[f'{name}']['hope_call_duty']
                info_days[0] = remove_list(info_days[0], members_info[f'{name}']['call_duty'])
                info_days[1] = remove_list(info_days[1], members_info[f'{name}']['call_duty'])
                info_days[2] = remove_list(info_days[2], members_info[f'{name}']['call_duty'])
                info_days[3] = remove_list(info_days[3], members_info[f'{name}']['call_duty'])

def decide_duty_members(members_info,info_days, interval, icu=True):
    if icu:
        icu_info_days = info_days.copy()
        vacation_days = (info_days[0]+[1]).copy()
        left_info_days = info_days.copy()
        weekend = False
        try_num = 0
        for days in left_info_days:
            left_days = days.copy()
            print(f'{left_days}に対してICU当直を決めます。')
            if days in vacation_days:
                weekend == True
            choosen_day = None
            highest_members = []
            potential_day = None
            while left_days:
                while not highest_members:
                    max_score = highest_duty_score(members_info, icu)[1]
                    highest_members = [name for name in members_info.keys()
                                      if (members_info[f'{name}']['duty_score'] == max_score and
                                        members_info[f'{name}']['no_icu_duty'] == False
                                          )]
                try_num +=1
                print('try number: ',try_num)
                if try_num >10:
                    max_score -=1
                    add_mem = [name for name in members_info.keys()
                                    if members_info[name]['duty_score'] == max_score
                                    and members_info[name]['no_icu_duty'] == False]
                    print(f'{add_mem}を候補者に追加します。')
                    highest_members = highest_members + add_mem
                    try_num = 0
                if try_num >15:
                    max_score = highest_duty_score(members_info, icu)[1]
                    try_num = 0

                sorted_mem = sorted(highest_members,
                                    key=lambda x: (len(unavailabledays_of_member(x, members_info, weekend))),
                                    reverse=True)



                print(f'{sorted_mem}の中で{potential_day}のICU当直を決めます')
                if sorted_mem:
                    for potential_mem in sorted_mem:
                        if left_days:
                            potential_day = random.choice(left_days)
                        print(f'{potential_mem}と{potential_day}日のicu当直をチェックします.')
                        print(f' Max_score: {max_score}, 実施回数：{try_num}')
                        if check_not_unavailable_day(potential_day, potential_mem, members_info) \
                            and check_not_within_interval(potential_day, potential_mem, members_info,interval, icu)\
                            and check_no_holidays(potential_mem, members_info, potential_day, icu_info_days, icu)\
                            and check_duty_num(members_info, potential_mem):
                            choosen_mem = potential_mem
                            choosen_day = potential_day
                            print(f'remove {choosen_day} from {left_days}')
                            left_days.remove(choosen_day)
                            print(f'残りの当直日は{left_days}')

                            if choosen_day not in info_days[3]:
                                members_info[choosen_mem]['no_holidays'] +=1
                            members_info[choosen_mem]['duty_num'] -=1
                            members_info[choosen_mem]['icu_duty'].append(choosen_day)
                            members_info[choosen_mem]['duty_score'] = count_down_duty_score(choosen_mem, choosen_day, members_info, left_info_days, icu)
                            highest_members = [name for name in members_info.keys()
                                                if members_info[name]['duty_score'] == max_score
                                                and members_info[name]['no_icu_duty'] == False]
                            print(f'{choosen_mem}は{choosen_day}日のICU当直ができます')
                            print('>>>>>>>>>'*10)
                            if not left_days:
                                break
                        else:
                            print(f'{potential_mem}は{potential_day}日のicu当直ができません.')
                            pass


            print('*************'*20)
            print('*************'*20)
        print('#$%&'*100)
    if icu == False:
        call_info_days = info_days.copy()
        vacation_days = (info_days[0]+[1]).copy()
        left_info_days = info_days.copy()
        weekend = False
        try_num = 0
        for days in left_info_days:
            left_days = days.copy()
            print(f'{left_days}に対してCALL当直を決めます。')
            if days in vacation_days:
                weekend == True
            choosen_day = None
            highest_members = []
            potential_day = None
            while left_days:
                while not highest_members:
                    max_score = highest_duty_score(members_info, icu)[1]
                    highest_members = [name for name in members_info.keys()
                                      if (members_info[f'{name}']['duty_score'] == max_score
                                          )]
                try_num +=1
                print('try number: ',try_num)
                if try_num >10:
                    max_score -=1
                    add_mem = [name for name in members_info.keys()
                                    if members_info[name]['duty_score'] == max_score]
                    print(f'{add_mem}を候補者に追加します。')
                    highest_members = highest_members + add_mem
                    try_num = 0
                if try_num >15:
                    max_score = highest_duty_score(members_info, icu)[1]
                    try_num = 0

                sorted_mem = sorted(highest_members,
                                    key=lambda x: (members_info[x]['no_icu_duty'],
                                                   len(unavailabledays_of_member(x, members_info, weekend))),
                                    reverse=True)

                print(f'{sorted_mem}の中で{potential_day}のCall当直を決めます')
                if sorted_mem:
                    for potential_mem in sorted_mem:
                        if left_days:
                            potential_day = random.choice(left_days)
                        print(f'{potential_mem}と{potential_day}日のCall当直をチェックします.')
                        print(f' Max_score: {max_score}, 実施回数：{try_num}')
                        if check_not_unavailable_day(potential_day, potential_mem, members_info) \
                            and check_not_within_interval(potential_day, potential_mem, members_info,interval, icu)\
                            and check_cardiac_pair(potential_mem, potential_day, members_info)\
                            and check_old_boy_pair(potential_mem, potential_day, members_info)\
                            and check_no_holidays(potential_mem, members_info, potential_day,call_info_days, icu)\
                            and check_duty_num(members_info, potential_mem):

                            choosen_mem = potential_mem
                            choosen_day = potential_day
                            print(f'remove {choosen_day} from {left_days}')
                            left_days.remove(choosen_day)
                            print(f'残りの当直日は{left_days}')
                            if choosen_day not in info_days[3]:
                                 members_info[choosen_mem]['no_holidays'] +=1
                            members_info[choosen_mem]['duty_num'] -=1
                            members_info[choosen_mem]['call_duty'].append(choosen_day)
                            members_info[choosen_mem]['duty_score'] = count_down_duty_score(choosen_mem, choosen_day,
                                                                                            members_info, left_info_days, icu)
                            highest_members = [name for name in members_info.keys()
                                                if members_info[name]['duty_score'] == max_score]
                            print(f'{choosen_mem}は{choosen_day}日のcall当直ができます')
                            print('>>>>>>>>>'*10)
                            if not left_days:
                                break
                        else:
                            print(f'{potential_mem}は{potential_day}日のcall当直ができません.')
                            pass


            print('*************'*20)
            print('*************'*20)
        print('#$%&'*100)




## 2.4 Function to check the rules for deciding duty members

In [136]:
#count down duty_score by duty class and duty day
def count_down_duty_score(name, day, members_info, info_days, icu= True):
    criteria_days = info_days.copy()
    score= 0
    duty_score = members_info[f'{name}']['duty_score']
    if icu:
        if day in criteria_days[0]:
            score = 7
        elif day in criteria_days[1]:
            score = 6
        elif day in criteria_days[2]:
            score = 7
        elif day in criteria_days[3]:
            score = 5
    else:
        if day in criteria_days[0]:
            score = 6
        elif day in criteria_days[1]:
            score = 6
        elif day in criteria_days[2]:
            score = 3
        elif day in criteria_days[3]:
            score = 3
    duty_score -= score
    print(f'day : {day}, score = {score}')
    print(f"{name}'s duty_score(after) : {duty_score}")
    return duty_score

#Check what is a highest duty_score of all members,
def highest_duty_score(members_info, icu = True):
    if icu:
        max_score = max(name['duty_score'] for name in members_info.values() if name['no_icu_duty'] != icu)
        keys = [key for key, value in members_info.items() if value['duty_score'] == max_score and value['no_icu_duty'] != icu]
    else:
        max_score = max(name['duty_score'] for name in members_info.values())
        keys = [key for key, value in members_info.items() if value['duty_score'] == max_score]
    return keys, int(max_score)


#Check the duty day if the day is contraindicated

#count the number of unavailable day on weekday or weekend
def unavailabledays_of_member(name, members_info, weekend = True):
    if weekend == True:
        left_days = remove_list(members_info[f'{name}']['unavailable_days'], weekdays)
    elif weekend ==False:
        left_days = remove_list(members_info[f'{name}']['unavailable_days'], vacation_days)

    return left_days

##Modulse for each contraindication

#Check if the day is in unavailable_day
def check_not_unavailable_day(choosen_day, choosen_member, members_info):
    print(choosen_day, choosen_member)
    if check_list(members_info[choosen_member]['unavailable_days'], [choosen_day]):
        print(f'{choosen_day} is unavailable day of {choosen_member}')
        return False
    else:
        return True


#Check if the day is within 3days after a previous duty day.
def check_not_within_interval(choosen_day, choosen_member, members_info, interval, icu = True):
    period = list(range(choosen_day-interval, choosen_day+interval+1))
    print(f'Days is {period}')
    if icu == True:
        print(f"{choosen_member}'s ICU duty is {members_info[choosen_member]['icu_duty']}")
        if check_list(members_info[choosen_member]['icu_duty'], period) :
            print(f"the {choosen_day} is contraindicated because within {interval} days from previous icu duty day")
            return False
        elif check_list(members_info[choosen_member]['call_duty'], period) :
            print(f"the {choosen_day} is contraindicated because within {interval} days from previous call duty day")
            return False
        else:
            return True
    if icu == False:
        print(f"{choosen_member}'s CALL duty is {members_info[choosen_member]['icu_duty']}")
        if check_list(members_info[choosen_member]['call_duty'], period) :
            print(f"the {choosen_day} is contraindicated because within {interval} days from previous call duty day")
            return False
        elif check_list(members_info[choosen_member]['icu_duty'], period) :
            print(f"the {choosen_day} is contraindicated because within {interval} days from previous icu duty day")
            return False
        else:
            return True
# def check_not_within_two(choosen_day, choosen_member, members_info, icu = True):
#     if icu == True:
#         if check_list(members_info[f'{choosen_member}']['icu_duty'], [choosen_day-3,choosen_day-2, choosen_day-1,choosen_day,choosen_day+1,choosen_day+2,choosen_day+3]) :
#             print(f'the {choosen_day} is contraindicated because within 2days from previous icu duty day')
#             return False
#         elif check_list(members_info[f'{choosen_member}']['call_duty'], [choosen_day-3,choosen_day-2, choosen_day-1,choosen_day,choosen_day+1,choosen_day+2]) :
#             print(f'the {choosen_day} is contraindicated because within 2days from previous call duty day')
#             return False
#         else:
#             return True
#     if icu == False:
#         if check_list(members_info[f'{choosen_member}']['call_duty'], [choosen_day-3,choosen_day-2, choosen_day-1,choosen_day,choosen_day+1,choosen_day+2]) :
#             print(f'the {choosen_day} is contraindicated because another call duty day is within 2days')
#             return False
#         elif check_list(members_info[f'{choosen_member}']['icu_duty'], [choosen_day-3,choosen_day-2, choosen_day-1,choosen_day,choosen_day+1,choosen_day+2]) :
#             print(f'the {choosen_day} is contraindicated because another icu duty day is within 2days')
#             return False
#         else:
#             return True


#Check if the confirmed duty team can handle cardiac surgery
def check_cardiac_pair(name, day, members_info):
    call_duty_name = name
    icu_duty_name = None
    for key, duty in members_info.items():
        if day in duty['icu_duty']:
            icu_duty_name = key
    if members_info[call_duty_name]['no_cardiac_surgery'] and members_info[icu_duty_name]['no_cardiac_surgery']:
        print(f'{icu_duty_name} of icu_duty of {day}  and  {call_duty_name} of call_duty is not able to handle Cardiac surgery.')
        return False
    else:
        print(f'{icu_duty_name} of icu_duty of {day}  and  {call_duty_name} of call_duty is able to handle Cardiac surgery.')
        return True

#Check if the number of no_holidays is over 2.
def check_no_holidays(name, members_info, potential_day, info_days, icu):
    if icu:
        if members_info[name]['no_holidays'] == 2 and potential_day not in info_days[3]:
            print("週末が2回以上えられたため当直の禁忌です")
            return False
        else:
            return True
    if icu == False:
        if members_info[name]['no_holidays'] == 2 and potential_day not in  info_days[3]:
            print("週末が2回以上えられたため当直の禁忌です")
            return False
        else:
            return True

def check_no_additional_duty(name, members_info):
    if members_info[name]["no_additional_duty"] ==1:
        return False
    else:
        return True

#Check if number of duty is over limitation of each members.
def check_duty_num(membs_info, name):
    if membs_info[name]["duty_num"] >=1:
        return True
    else:
        print(f"Total number of duty is over 5 days. So {name} can't be assigned to another duty anymore")
        return False

#Check if both members of pair are old boy
def check_old_boy_pair(name, day, members_info):
    call_duty_name = name
    icu_duty_name = None
    for key, duty in members_info.items():
        if day in duty['icu_duty']:
            icu_duty_name = key
    if members_info[call_duty_name]['old_boy'] and members_info[icu_duty_name]['old_boy']:
        print(f'Both {icu_duty_name} and {call_duty_name} at {day} day  is OLD BOY.')
        return False
    else:
        print(f'Either {icu_duty_name} or {call_duty_name} at {day} is not OLD BOY.')
        return True

## 2.4 Functions to make a form of dataframe, date table, calendar and excel file

In [137]:
#Make a dataframe of duty days of all members
def make_schedule_df(members_info):
    confirmed_schedule = pd.DataFrame(index=range(1,period+1), columns = ['icu_duty','call_duty'])
    for name in members_info.keys():
        for n in members_info[f'{name}']['icu_duty']:
            confirmed_schedule['icu_duty'][n] = name
    for name in members_info.keys():
        for n in members_info[f'{name}']['call_duty']:
            confirmed_schedule['call_duty'][n] = name
    print(confirmed_schedule)
    return confirmed_schedule

#Make a dataframe as a shape of calendar
def make_calendar(table, year, month, start, end):
    df = table
    # Create a date range for January 2024
    dates = pd.date_range(start=start, end=end)

    # Assign the dates to the dataframe
    df['date'] = dates

    # Create a calendar
    cal = calendar.monthcalendar(year, month)

    # Create a new dataframe for the calendar
    calendar_df = pd.DataFrame(index=pd.MultiIndex.from_product([['1st week', '2nd week', '3rd week', '4th week', '5th week','6th week'], ['day','icu_duty', 'call_duty']], names=['Week', 'Duty']), columns=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

    # Fill the calendar dataframe with the duties
    for i, week in enumerate(cal):
        for j, day in enumerate(week):
            if day != 0:  # Calendar's month days are represented as 0
                duty = df[df['date'].dt.day == day]
                if day in vacation_days:
                    calendar_df.loc[(f'{i+1}st week', 'day'), calendar_df.columns[j]] = f'*{day}'
                else:
                    calendar_df.loc[(f'{i+1}st week', 'day'), calendar_df.columns[j]] = day
                calendar_df.loc[(f'{i+1}st week', 'icu_duty'), calendar_df.columns[j]] = duty['icu_duty'].values[0]
                calendar_df.loc[(f'{i+1}st week', 'call_duty'), calendar_df.columns[j]] = duty['call_duty'].values[0]

    # Drop the weeks where all duties are NaN
    calendar_df.dropna(how='all', inplace=True)

    # Print the calendar
    print(calendar_df)
    return calendar_df

## 2.5 Functions to simplify function

In [138]:
#remove elements in list2 from list 1
def remove_list(list1, list2):
    set2 = set(list2)
    return [element for element in list1 if element not in set2]

#check if there is element of list2 in list1
def check_list(list1, list2):
    intersection = [n for n in list1 if n in list2]
    print(intersection)
    if len(intersection) == 0:
        return False
    else:
        return True

#Return elements shared between list1 and list2

def shared_elements(list1, list2):
    common_elements = set(list1).intersection(set(list2))
    return list(common_elements)


## 2.6 Deprecated Functions

#  3. 만들고자 하는 년월의 정보 로딩

## 3.1 Make a dataframe from excel file And Take a information

In [139]:
#Take a information of Year and Month that user wants to use
df_date = pd.read_excel(path, sheet_name = 'hoped_day', nrows = 1, header = None)
year = df_date.iloc[0,1]
month = df_date.iloc[0,3]
_, month_range = calendar.monthrange(year, month)
print(f"This schedule is of {year}/{month}")

This schedule is of 2024/6


In [140]:
#Load Excel file and take a information of date.
df_hoped_day= pd.read_excel(path, sheet_name='hoped_day',header = 1,nrows=month_range)
start_index = start_index = df_hoped_day.index[df_hoped_day['日付'].isna()].min()
start_column = df_hoped_day.columns[2]
end_column = df_hoped_day.columns[-1]
num_memb = len(df_hoped_day.columns)-1
name_of_memb = list(df_hoped_day.columns[1:])

columns_range = df_hoped_day.loc[:, start_column:end_column]
# Survey unique data within the specified range of columns
unique_values = columns_range.stack().unique()

df_hoped_day

,日付,祝日,北,清水と,荒井,柴田,井上,上田,一澤,西村,...,小出,竹田,松村,寺西,清水ち,髙田,池田,久保,李,竹山
0,2024-06-01,NaN,NaN,NaN,X,NaN,学会,X,学会,NaN,...,NaN,X,NaN,NaN,NaN,X,X,NaN,NaN,NaN
1,2024-06-02,NaN,NaN,NaN,X,NaN,学会,NaN,CALL,NaN,...,NaN,X,NaN,NaN,NaN,X,X,NaN,NaN,NaN
2,2024-06-03,NaN,NaN,NaN,NaN,NaN,NaN,ICU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN
3,2024-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-06-05,NaN,NaN,X,X,X,X,NaN,X,NaN,...,NaN,NaN,NaN,X,NaN,NaN,X,X,X,NaN
5,2024-06-06,麻酔科学会,学会,NaN,NaN,学会,NaN,学会,NaN,X,...,X,学会,NaN,NaN,NaN,X,NaN,NaN,学会,X
6,2024-06-07,麻酔科学会,本院,NaN,NaN,NaN,学会,学会,学会,NaN,...,NaN,NaN,NaN,学会,NaN,NaN,学会,学会,学会,NaN
7,2024-06-08,麻酔科学会,CALL,NaN,NaN,NaN,NaN,学会,X,NaN,...,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN
8,2024-06-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,...,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2024-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,~17,NaN,...,NaN,１日休,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN


In [141]:
select_class = {1 : ['外勤', 'X', '１日休', '1日休み', '~16', '~17', '~13', '学会',"出張",'~15', '本院',"夏休"],
                2 : ['ICU'],
                3 : ['CALL']}

#Value and keys can be exchanged by under code.
value_to_key = {val : key for key, values in select_class.items() for val in values}
#By dictionary, transfor a Key to value and make type int. And if there is no key in dict, np.nan will be 0.
df_trans_hoped_day = df_hoped_day.loc[:, start_column:end_column].applymap(lambda x : int(value_to_key.get(x,0)))
df_trans_hoped_day.index= df_hoped_day["日付"]
df_trans_hoped_day

/var/folders/h5/jyq7r4dd38l9x866mmggyx280000gn/T/ipykernel_2952/141328075.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_trans_hoped_day = df_hoped_day.loc[:, start_column:end_column].applymap(lambda x : int(value_to_key.get(x,0)))


,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,...,小出,竹田,松村,寺西,清水ち,髙田,池田,久保,李,竹山
日付,,,,,,,,,,,,,,,,,,,,,
2024-06-01,0,0,1,0,1,1,1,0,0,2,...,0,1,0,0,0,1,1,0,0,0
2024-06-02,0,0,1,0,1,0,3,0,0,0,...,0,1,0,0,0,1,1,0,0,0
2024-06-03,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2024-06-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2024-06-05,0,1,1,1,1,0,1,0,0,1,...,0,0,0,1,0,0,1,1,1,0
2024-06-06,1,0,0,1,0,1,0,1,1,0,...,1,1,0,0,0,1,0,0,1,1
2024-06-07,1,0,0,0,1,1,1,0,0,1,...,0,0,0,1,0,0,1,1,1,0
2024-06-08,3,0,0,0,0,1,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2024-06-09,0,0,0,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [142]:
#Load excel file and take a information of extra_info
df_extra_info= pd.read_excel(path, sheet_name='extra_info',header = 0)
df_extra_info

,date,extra_holidays,no_second_hospital
0,2024-06-01,NaN,NaN
1,2024-06-02,NaN,NaN
2,2024-06-03,NaN,NaN
3,2024-06-04,NaN,NaN
4,2024-06-05,NaN,NaN
5,2024-06-06,NaN,NaN
6,2024-06-07,NaN,NaN
7,2024-06-08,NaN,NaN
8,2024-06-09,NaN,NaN
9,2024-06-10,NaN,NaN


In [143]:
#Take a information of mem_info(outside work, availability of duty,icu_duty and cardiac surgery) from excel file
df_mem_info = pd.read_excel(path, sheet_name='mem_info',header = 0)
df_mem_info

,name,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,...,小出,竹田,松村,寺西,清水ち,髙田,池田,久保,李,竹山
0,中津,0,1,0,0,0,0,0,0,1,...,0,1,1,0,1,0,1,1,1,1
1,千船,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,南大阪,0,0,0,0,0,0,1,1,0,...,0,1,1,0,1,0,0,0,0,0
3,貝塚,0,0,1,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
4,伊丹,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,外勤回数,2,2,2,2,2,0,2,3,3,...,3,0,3,0,3,0,3,3,3,3
6,第２回数,0,0,0,0,0,1,1,1,1,...,1,1,1,0,1,0,1,1,1,1
7,no_icu_duty,1,1,1,0,1,0,1,0,0,...,1,0,1,0,1,0,0,0,0,1
8,no_cardiac_surgery,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,no_duty,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0,0,1


In [144]:
#Delete members of no_duty from df_trans_hoped_day
no_duty = list(df_mem_info.columns[(df_mem_info[df_mem_info['name'] == 'no_duty'] == 1).iloc[0]])
df_trans_hoped_day = df_trans_hoped_day.drop(columns = no_duty)
df_trans_hoped_day

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
日付,,,,,,,,,,,,,,,,,,
2024-06-01,0,0,1,0,1,1,1,0,0,2,0,1,1,0,1,1,0,0
2024-06-02,0,0,1,0,1,0,3,0,0,0,0,1,1,0,1,1,0,0
2024-06-03,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,1,0,0
2024-06-04,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2024-06-05,0,1,1,1,1,0,1,0,0,1,1,0,0,1,0,1,1,1
2024-06-06,1,0,0,1,0,1,0,1,1,0,0,1,1,0,1,0,0,1
2024-06-07,1,0,0,0,1,1,1,0,0,1,0,1,0,1,0,1,1,1
2024-06-08,3,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1
2024-06-09,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0


In [145]:
#Take a information of outpatient duty doctor from excel file
df_outpatient = pd.read_excel(path, sheet_name='outpatient',header = 0)
df_outpatient

/opt/anaconda3/lib/python3.11/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


,name,mon,tue,wed,thu,fri
0,pain_outpatient,一澤,池田,松村,井上,井上
1,preop_outpatient,北,猪原,寺西,竹田,一澤


In [146]:
#Take a information of available weekdays of outside work
df_outside_work = pd.read_excel(path, sheet_name='outside_work',header = 0)
df_outside_work

,name,mon,tue,wed,thu,fri
0,中津,1,1,0,1,1
1,千船,1,0,0,0,0
2,南大阪,0,1,1,0,0
3,貝塚,0,0,1,0,1
4,伊丹,0,0,0,1,0
5,第二,1,1,1,0,1


--------------------------------------------------------------------------------------------------------------------------------------------

## 3.2 Make a applied information based on loaded data

In [147]:
#Get a information from excel file by using function
holidays = get_holidays_from_api(year)
is_next_holiday = check_first_day_of_next_month(year, month, holidays)
extra_holiday = get_extra_holidays(df_extra_info)
#Define year and month. And monthrange can be achievied by calendar library.
period = calendar.monthrange(year, month)[1]
start = datetime(year, month, 1).strftime('%Y-%m-%d')
end = datetime(year,month, period).strftime('%Y-%m-%d')

#Make a list of no_cardiac_surgery and no_icu_duty and so on of all members
no_cardiac_surgery, no_icu_duty, no_additional_duty, old_boy = list_memb_skill(df_mem_info)

Japanese Holidays of 2024 : 

the First day of Next Month is NOT holiday
Old boys are ['北', '清水と', '荒井', '柴田']
週末の当直ができる人員数：18
Old boyで週末当直1回にする人員数：4
今月に週末当直の総数：28
holiday_availability is 4
Required number of twice old boy is 2
Members of old boys take twice of duty on holiday ['柴田', '荒井']


## 3.3 Classify days based on calendar and extra holidays

In [149]:
#Classify days to weekdays and holidays
weekdays, vacation_days = distinct_days(year, month, extra_holiday, holidays)
print("休日 :", vacation_days)
print("平日 ;", weekdays)

#Classify days by criterias
# last_weekday, not_last_weekdays = last_weekdays(weekdays, vacation_days, period, is_next_holiday)
# not_last_vacation_days, last_vacation_days = not_last_holiday(vacation_days, period, is_next_holiday)

#List of lists of classified days. Order is weight decreased
info_days = list(inform_days(weekdays, vacation_days, period, is_next_holiday))
# preday, weekdays = get_preday_of_holiday(weekdays, vacation_days)
print('普通の平日 : ',info_days[3] )
print('休日の前日の平日 : ', info_days[2])
print('翌日平日である休日 : ', info_days[1])
print('翌日が休日である休日 : ', info_days[0])

There is NOT extra holiday at this month.
Holidays of this month : [1, 2, 8, 9, 15, 16, 22, 23, 29, 30]
Weekdays of this month : [3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28]
休日 : [1, 2, 8, 9, 15, 16, 22, 23, 29, 30]
平日 ; [3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28]
1 day is NOT a last day of vacation days
[1]
2 day is a last day of vacation_days
8 day is NOT a last day of vacation days
[1, 8]
9 day is a last day of vacation_days
15 day is NOT a last day of vacation days
[1, 8, 15]
16 day is a last day of vacation_days
22 day is NOT a last day of vacation days
[1, 8, 15, 22]
23 day is a last day of vacation_days
29 day is NOT a last day of vacation days
[1, 8, 15, 22, 29]
30 day is a last day of vacation_days
3 day is not a last day of weekdays
4 day is not a last day of weekdays
5 day is not a last day of weekdays
6 day is not a last day of weekdays
7 day is a last day of weekdays
10 day is not a last day of weekdays
11 day 

# 4. 각 회사의 직원들의 기본 정보와 그눔 희망날짜, 피하고 싶은 날짜, 가능한 능력에 대해 직접 입력

## 4.1 기본 정보 입력 및 입력 된 정보를 DataFrame으로 확인

# 5. 당직일자를 정하기

> 1) 희망 당직날짜부터 배정

> 2) 주말 및 휴일 당직 배정

> 3) duty_score, 희망 당직일, 피하고 싶은 당직일 등 문제가 없는지 최종확인


## 5.0 당직의 규칙 정하기(new_version)

>1) icu당직을 한 경우, 다음날을 포함 2일간은 icu 및 call당직을 하지 않는다.

>2)call당직을 한 경우, icu당직은 다음날에 하지 않으며, 다음 call 당직은 최소 2일간은 하지 않음.

>3)call 당직자와 icu 당직자 중 한명은 심장수술 대응이 가능해야 함.

>3) 희망일을 우선 배정하지만, 겹치지 않는 다는 전제에 함. 그렇기 때문에 당직날을 정하기 전에 당사자간 협의하도록 한다.

>4) 휴일 icu당직은 기본 1일로 정하되, 사람이 부족한 경우에는 2일을 할 수 있음.

>) 평일 icu당직은 icu주말당직을 하지 않는 사람은 기본 2일을 하고, 추가로 필요하다면, 휴일 icu당직자에서 duty_score가 높은 사람 중에서 랜덤으로 선택.

>5) 휴일 call 당직은 icu당직을 하지 않는 멤버 그 다음은 duty_score가 낮은 사람을 배정

>> 5-1) 特定の人は週末Call_dutyをしないかも

>6) 평일 call 당직을 duty_score가 높은 사람부터 순차적으로 배정

>7) 모든 과정은 랜덤으로 배정하고, 금기사항을 지킬 수 없는 사항이면, 에러메시지를 표시하고, 처음부터 다시시작.

>8) Weight(6 classification)

>>: 연휴 첫날부터 연휴 마지막 날의 전날icu > 연휴 마지막날icu > 휴일 전날의 평일icu > 나머지 평일icu > 휴일 call당직 > 평일 call당직

>9) 주말에 icu나 call　당직은 한달에 2번만 들어가게 만들기

>10) 되도록이면, 주말의 icu 혹은 call당직으로 주말이 망가지는건 2번까지만 허용.

>
> 1)Deciding duty members on weekend and holydays

> 2)First, Decide icu_duty members

> 3)randomly distribution -> check unavailable_day -> check_within_2days after previous_duty

> 4)Second, Decide call_duty members

> 5)randomly distribution -> check unavailable_day -> check_within 2days after previous_duty

> 6)Third, Check if the duty pair can be able to handle cardiac surgery.

> 7)if they are unable to handle cardiac surgery, Restart alogorithm of deciding weekend duty members.

> 8)if there is 5 tries for deciding and we can decide members by algorithm, make a Error message to user.

## 5.1 Making a Dataframe based on collected information

In [162]:
members_info = make_members_info(df_trans_hoped_day, df_mem_info)
members_df = pd.DataFrame(members_info)

#Check if this month requires old boy to do holiday duty twice. And manipulate holiday duty count of holiday duty of old boys
members_df = select_twice_old_boy(df_mem_info = df_mem_info, 
                                  info_days = info_days, 
                                  members_df = members_df,
                                  old_boy = old_boy)
#Apply infomation about no_holidas to members_info based on members_df
members_info = members_df.to_dict(orient="dict")

origin_members_info = members_info.copy()

心臓手術非対応　:  ['北', '井上', '上田', '猪原', '小山', '竹山']
ICU当直非対応 :  ['北', '清水と', '荒井', '井上', '一澤', '猪原', '小出', '松村', '清水ち', '竹山']
指定Duty以外は追加無し :  ['北', '井上', '上田', '猪原', '竹山']
OLD BOY :  ['北', '清水と', '荒井', '柴田']
Old boys are ['北', '清水と', '荒井', '柴田']
週末の当直ができる人員数：18
Old boyで週末当直1回にする人員数：4
今月に週末当直の総数：28
holiday_availability is 4
Required number of twice old boy is 2
Members of old boys take twice of duty on holiday ['柴田', '荒井']


/var/folders/h5/jyq7r4dd38l9x866mmggyx280000gn/T/ipykernel_2952/3709963533.py:42: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  members_info[name]['duty_num']= int(mem_info.loc[mem_info['name']=='duty_num',name])


In [163]:
a = members_df.to_dict(orient="dict")
print(a)

{'北': {'no_icu_duty': True, 'no_cardiac_surgery': True, 'no_additional_duty': True, 'old_boy': True, 'unavailable_days': [6, 7, 20, 21, 22, 23], 'hope_icu_duty': [], 'hope_call_duty': [8], 'icu_duty': [], 'call_duty': [], 'duty_score': -100, 'no_holidays': 1, 'duty_num': 1}, '清水と': {'no_icu_duty': True, 'no_cardiac_surgery': False, 'no_additional_duty': False, 'old_boy': True, 'unavailable_days': [5, 22, 23, 28], 'hope_icu_duty': [], 'hope_call_duty': [], 'icu_duty': [], 'call_duty': [], 'duty_score': 5, 'no_holidays': 1, 'duty_num': 3}, '荒井': {'no_icu_duty': True, 'no_cardiac_surgery': False, 'no_additional_duty': False, 'old_boy': True, 'unavailable_days': [1, 2, 5, 15, 20, 21, 22], 'hope_icu_duty': [], 'hope_call_duty': [], 'icu_duty': [], 'call_duty': [], 'duty_score': 5, 'no_holidays': 0, 'duty_num': 3}, '柴田': {'no_icu_duty': False, 'no_cardiac_surgery': False, 'no_additional_duty': False, 'old_boy': True, 'unavailable_days': [5, 6], 'hope_icu_duty': [], 'hope_call_duty': [], 'icu

In [153]:
# icu_info_days = info_days.copy()
# call_info_days = info_days.copy()
# hoped_duty(members_info, icu_info_days, icu = True)
# hoped_duty(members_info, call_info_days, icu = False)


In [164]:
df = pd.DataFrame(members_info)
df

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
no_icu_duty,True,True,True,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False
no_cardiac_surgery,True,False,False,False,True,True,False,False,False,True,False,True,False,False,False,False,False,False
no_additional_duty,True,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False
old_boy,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
unavailable_days,"[6, 7, 20, 21, 22, 23]","[5, 22, 23, 28]","[1, 2, 5, 15, 20, 21, 22]","[5, 6]","[1, 2, 5, 7, 14, 15, 16, 22, 28]","[1, 6, 7, 8, 12, 13, 14, 15, 16, 21, 28]","[1, 5, 7, 8, 9, 10, 12, 14, 15, 16, 22, 23, 26...","[6, 16]",[6],"[5, 7, 9, 21, 27]",[5],"[1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 17, 18, 24, 2...","[1, 2, 6, 8, 9, 10, 15, 16, 29]","[5, 7, 14, 15, 16, 29]","[1, 2, 6, 14, 15, 21, 22]","[1, 2, 3, 5, 7, 10, 17, 21, 22, 23, 24, 28]","[5, 7]","[5, 6, 7, 8, 20, 21, 22]"
hope_icu_duty,[],[],[],[],[],"[3, 17]",[],[],[],"[1, 16, 22, 30]",[],[],[],[],[19],[26],[],[]
hope_call_duty,[8],[],[],[],[29],[],[2],[],[],[],[],[],[],[],[],[],[],[]
icu_duty,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
call_duty,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
duty_score,-100,5,5,11,-100,-100,5,11,11,-100,11,11,11,11,11,11,11,11


## 5.2 Apply data of hoped day to members_info dataframe

In [165]:
#희망일자부터 처리!
#만약에 겹치면, 일정표 짜기전에 서로 상의해서 한명만 남기도록 하기!

#if there is a with to do duty on specified days, first of all, distribute duty days members wished
icu_info_days = info_days.copy()
icu_info_days[0] = icu_info_days[0]
icu_info_days[1] = icu_info_days[1]
icu_info_days[2] = icu_info_days[2]
icu_info_days[3] = icu_info_days[3]

call_info_days = info_days.copy()
call_info_days[0] = call_info_days[0]
call_info_days[1] = call_info_days[1]
call_info_days[2] = call_info_days[2]
call_info_days[3] = call_info_days[3]

for name in members_info.keys():
    if members_info[name]['hope_icu_duty'] :
        for day in members_info[name]['hope_icu_duty']:
            members_info[name]['duty_score'] = count_down_duty_score(name,
                                                                            day,
                                                                            members_info,
                                                                            info_days,
                                                                            icu=True)
            if day not in info_days[3]:
                members_info[name]['no_holidays'] +=1
            members_info[name]['duty_num'] -=1


        members_info[name]['icu_duty'] = members_info[name]['hope_icu_duty']
        members_info[name]['hope_icu_duty'] = []
        icu_info_days[0] = remove_list(icu_info_days[0], members_info[name]['icu_duty'])
        icu_info_days[1] = remove_list(icu_info_days[1], members_info[name]['icu_duty'])
        icu_info_days[2] = remove_list(icu_info_days[2], members_info[name]['icu_duty'])
        icu_info_days[3] = remove_list(icu_info_days[3], members_info[name]['icu_duty'])
for name in members_info.keys():
    if members_info[name]['hope_call_duty'] :
        for day in members_info[name]['hope_call_duty']:
            members_info[name]['duty_score'] = count_down_duty_score(name,
                                                                            day,
                                                                            members_info,
                                                                            info_days,
                                                                            icu=False)
            if day in (info_days[0]+ info_days[1]):
                members_info[name]['no_holidays'] +=1
            members_info[name]['duty_num'] -=1
        members_info[name]['call_duty'] = members_info[name]['hope_call_duty']
        members_info[name]['hope_call_duty'] = []
        call_info_days[0] = remove_list(call_info_days[0], members_info[name]['call_duty'])
        call_info_days[1] = remove_list(call_info_days[1], members_info[name]['call_duty'])
        call_info_days[2] = remove_list(call_info_days[2], members_info[name]['call_duty'])
        call_info_days[3] = remove_list(call_info_days[3], members_info[name]['call_duty'])


day : 3, score = 5
上田's duty_score(after) : -105
day : 17, score = 5
上田's duty_score(after) : -110
day : 1, score = 7
猪原's duty_score(after) : -107
day : 16, score = 6
猪原's duty_score(after) : -113
day : 22, score = 7
猪原's duty_score(after) : -120
day : 30, score = 6
猪原's duty_score(after) : -126
day : 19, score = 5
髙田's duty_score(after) : 6
day : 26, score = 5
池田's duty_score(after) : 6
day : 8, score = 6
北's duty_score(after) : -106
day : 29, score = 6
井上's duty_score(after) : -106
day : 2, score = 6
一澤's duty_score(after) : -1


In [166]:
df = pd.DataFrame(members_info)
df

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
no_icu_duty,True,True,True,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False
no_cardiac_surgery,True,False,False,False,True,True,False,False,False,True,False,True,False,False,False,False,False,False
no_additional_duty,True,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False
old_boy,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
unavailable_days,"[6, 7, 20, 21, 22, 23]","[5, 22, 23, 28]","[1, 2, 5, 15, 20, 21, 22]","[5, 6]","[1, 2, 5, 7, 14, 15, 16, 22, 28]","[1, 6, 7, 8, 12, 13, 14, 15, 16, 21, 28]","[1, 5, 7, 8, 9, 10, 12, 14, 15, 16, 22, 23, 26...","[6, 16]",[6],"[5, 7, 9, 21, 27]",[5],"[1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 17, 18, 24, 2...","[1, 2, 6, 8, 9, 10, 15, 16, 29]","[5, 7, 14, 15, 16, 29]","[1, 2, 6, 14, 15, 21, 22]","[1, 2, 3, 5, 7, 10, 17, 21, 22, 23, 24, 28]","[5, 7]","[5, 6, 7, 8, 20, 21, 22]"
hope_icu_duty,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
hope_call_duty,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
icu_duty,[],[],[],[],[],"[3, 17]",[],[],[],"[1, 16, 22, 30]",[],[],[],[],[19],[26],[],[]
call_duty,[8],[],[],[],[29],[],[2],[],[],[],[],[],[],[],[],[],[],[]
duty_score,-106,5,5,11,-106,-110,-1,11,11,-126,11,11,11,11,6,6,11,11


## 5.3 Assign ICU duty

In [167]:
decide_duty_members(members_info, icu_info_days, interval = 3, icu= True)

[8, 15, 29]に対してICU当直を決めます。
try number:  1
['小山', '李', '寺西', '柴田', '竹田', '久保', '西村', '孫', '河合']の中でNoneのICU当直を決めます
小山と8日のicu当直をチェックします.
 Max_score: 11, 実施回数：1
8 小山
[8]
8 is unavailable day of 小山
小山は8日のicu当直ができません.
李と29日のicu当直をチェックします.
 Max_score: 11, 実施回数：1
29 李
[]
Days is [26, 27, 28, 29, 30, 31, 32]
李's ICU duty is []
[]
[]
remove 29 from [8, 15, 29]
残りの当直日は[8, 15]
day : 29, score = 7
李's duty_score(after) : 4
李は29日のICU当直ができます
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
寺西と15日のicu当直をチェックします.
 Max_score: 11, 実施回数：1
15 寺西
[15]
15 is unavailable day of 寺西
寺西は15日のicu当直ができません.
柴田と15日のicu当直をチェックします.
 Max_score: 11, 実施回数：1
15 柴田
[]
Days is [12, 13, 14, 15, 16, 17, 18]
柴田's ICU duty is []
[]
[]
remove 15 from [8, 15]
残りの当直日は[8]
day : 15, score = 7
柴田's duty_score(after) : 4
柴田は15日のICU当直ができます
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
竹田と8日のicu当直をチェックします.
 Max_score: 11, 実施回数：1
8 竹田
[8]
8 is unavailab

In [168]:
df = pd.DataFrame(members_info)
df

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
no_icu_duty,True,True,True,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False
no_cardiac_surgery,True,False,False,False,True,True,False,False,False,True,False,True,False,False,False,False,False,False
no_additional_duty,True,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False
old_boy,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
unavailable_days,"[6, 7, 20, 21, 22, 23]","[5, 22, 23, 28]","[1, 2, 5, 15, 20, 21, 22]","[5, 6]","[1, 2, 5, 7, 14, 15, 16, 22, 28]","[1, 6, 7, 8, 12, 13, 14, 15, 16, 21, 28]","[1, 5, 7, 8, 9, 10, 12, 14, 15, 16, 22, 23, 26...","[6, 16]",[6],"[5, 7, 9, 21, 27]",[5],"[1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 17, 18, 24, 2...","[1, 2, 6, 8, 9, 10, 15, 16, 29]","[5, 7, 14, 15, 16, 29]","[1, 2, 6, 14, 15, 21, 22]","[1, 2, 3, 5, 7, 10, 17, 21, 22, 23, 24, 28]","[5, 7]","[5, 6, 7, 8, 20, 21, 22]"
hope_icu_duty,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
hope_call_duty,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
icu_duty,[],[],[],"[15, 25]",[],"[3, 17]",[],"[2, 24, 11]","[21, 5]","[1, 16, 22, 30]","[14, 10]","[23, 13]","[28, 12]","[9, 18]","[19, 7]","[26, 20, 6]","[8, 27]","[29, 4]"
call_duty,[8],[],[],[],[29],[],[2],[],[],[],[],[],[],[],[],[],[],[]
duty_score,-106,5,5,-1,-106,-110,-1,-5,-1,-126,-1,0,-1,0,-1,-4,-1,-1


## 5.4 Assign Call duty

In [169]:
decide_duty_members(members_info, call_info_days, interval = 2, icu = False)

[1, 15, 22]に対してCALL当直を決めます。
try number:  1
['荒井', '清水と']の中でNoneのCall当直を決めます
荒井と15日のCall当直をチェックします.
 Max_score: 5, 実施回数：1
15 荒井
[15]
15 is unavailable day of 荒井
荒井は15日のcall当直ができません.
清水とと15日のCall当直をチェックします.
 Max_score: 5, 実施回数：1
15 清水と
[]
Days is [13, 14, 15, 16, 17]
清水と's CALL duty is []
[]
[]
柴田 of icu_duty of 15  and  清水と of call_duty is able to handle Cardiac surgery.
Both 柴田 and 清水と at 15 day  is OLD BOY.
清水とは15日のcall当直ができません.
try number:  2
['荒井', '清水と']の中で15のCall当直を決めます
荒井と15日のCall当直をチェックします.
 Max_score: 5, 実施回数：2
15 荒井
[15]
15 is unavailable day of 荒井
荒井は15日のcall当直ができません.
清水とと22日のCall当直をチェックします.
 Max_score: 5, 実施回数：2
22 清水と
[22]
22 is unavailable day of 清水と
清水とは22日のcall当直ができません.
try number:  3
['荒井', '清水と']の中で22のCall当直を決めます
荒井と22日のCall当直をチェックします.
 Max_score: 5, 実施回数：3
22 荒井
[22]
22 is unavailable day of 荒井
荒井は22日のcall当直ができません.
清水とと15日のCall当直をチェックします.
 Max_score: 5, 実施回数：3
15 清水と
[]
Days is [13, 14, 15, 16, 17]
清水と's CALL duty is []
[]
[]
柴田 of icu_duty of 15  and  清水と of call_dut

In [170]:
df = pd.DataFrame(members_info)
df

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
no_icu_duty,True,True,True,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False
no_cardiac_surgery,True,False,False,False,True,True,False,False,False,True,False,True,False,False,False,False,False,False
no_additional_duty,True,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False
old_boy,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
unavailable_days,"[6, 7, 20, 21, 22, 23]","[5, 22, 23, 28]","[1, 2, 5, 15, 20, 21, 22]","[5, 6]","[1, 2, 5, 7, 14, 15, 16, 22, 28]","[1, 6, 7, 8, 12, 13, 14, 15, 16, 21, 28]","[1, 5, 7, 8, 9, 10, 12, 14, 15, 16, 22, 23, 26...","[6, 16]",[6],"[5, 7, 9, 21, 27]",[5],"[1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 17, 18, 24, 2...","[1, 2, 6, 8, 9, 10, 15, 16, 29]","[5, 7, 14, 15, 16, 29]","[1, 2, 6, 14, 15, 21, 22]","[1, 2, 3, 5, 7, 10, 17, 21, 22, 23, 24, 28]","[5, 7]","[5, 6, 7, 8, 20, 21, 22]"
hope_icu_duty,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
hope_call_duty,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
icu_duty,[],[],[],"[15, 25]",[],"[3, 17]",[],"[2, 24, 11]","[21, 5]","[1, 16, 22, 30]","[14, 10]","[23, 13]","[28, 12]","[9, 18]","[19, 7]","[26, 20, 6]","[8, 27]","[29, 4]"
call_duty,[8],"[1, 6, 12]","[16, 9, 3]",[30],[29],[],"[2, 18, 11, 25]",[5],"[28, 24]",[],"[27, 17]","[20, 26]","[7, 19]","[22, 4]",[23],[14],"[21, 13]","[15, 10]"
duty_score,-106,-7,-10,-7,-106,-110,-10,-8,-7,-126,-7,-6,-7,-9,-7,-7,-7,-10


## 5.5 Make a Dataframe based on collected data and Assigning duty day by algorithm.

In [171]:
for name in members_info.keys():
    if origin_members_info[f'{name}']['hope_icu_duty']:
        members_info[f'{name}']['hope_icu_duty'] = origin_members_info[f'{name}']['hope_icu_duty'] #+ members_info[f'{name}']['hope_icu_duty']
    if origin_members_info[f'{name}']['hope_call_duty']:
        members_info[f'{name}']['hope_call_duty'] = origin_members_info[f'{name}']['hope_call_duty'] #+ members_info[f'{name}']['hope_icu_duty']
    members_info[f'{name}']['icu_duty'] = sorted(members_info[f'{name}']['icu_duty'])
    members_info[f'{name}']['call_duty'] = sorted(members_info[f'{name}']['call_duty'])

df = pd.DataFrame(members_info)
df

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
no_icu_duty,True,True,True,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False
no_cardiac_surgery,True,False,False,False,True,True,False,False,False,True,False,True,False,False,False,False,False,False
no_additional_duty,True,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False
old_boy,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
unavailable_days,"[6, 7, 20, 21, 22, 23]","[5, 22, 23, 28]","[1, 2, 5, 15, 20, 21, 22]","[5, 6]","[1, 2, 5, 7, 14, 15, 16, 22, 28]","[1, 6, 7, 8, 12, 13, 14, 15, 16, 21, 28]","[1, 5, 7, 8, 9, 10, 12, 14, 15, 16, 22, 23, 26...","[6, 16]",[6],"[5, 7, 9, 21, 27]",[5],"[1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 17, 18, 24, 2...","[1, 2, 6, 8, 9, 10, 15, 16, 29]","[5, 7, 14, 15, 16, 29]","[1, 2, 6, 14, 15, 21, 22]","[1, 2, 3, 5, 7, 10, 17, 21, 22, 23, 24, 28]","[5, 7]","[5, 6, 7, 8, 20, 21, 22]"
hope_icu_duty,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
hope_call_duty,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
icu_duty,[],[],[],"[15, 25]",[],"[3, 17]",[],"[2, 11, 24]","[5, 21]","[1, 16, 22, 30]","[10, 14]","[13, 23]","[12, 28]","[9, 18]","[7, 19]","[6, 20, 26]","[8, 27]","[4, 29]"
call_duty,[8],"[1, 6, 12]","[3, 9, 16]",[30],[29],[],"[2, 11, 18, 25]",[5],"[24, 28]",[],"[17, 27]","[20, 26]","[7, 19]","[4, 22]",[23],[14],"[13, 21]","[10, 15]"
duty_score,-106,-7,-10,-7,-106,-110,-10,-8,-7,-126,-7,-6,-7,-9,-7,-7,-7,-10


# 6. Show tables

## 6.1 Table of List type of Duty days

In [172]:
table = make_schedule_df(members_info)

   icu_duty call_duty
1        猪原       清水と
2        西村        一澤
3        上田        荒井
4         李        寺西
5         孫        西村
6        池田       清水と
7        髙田        竹田
8        久保         北
9        寺西        荒井
10       河合         李
11       西村        一澤
12       竹田       清水と
13       小山        久保
14       河合        池田
15       柴田         李
16       猪原        荒井
17       上田        河合
18       寺西        一澤
19       髙田        竹田
20       池田        小山
21        孫        久保
22       猪原        寺西
23       小山        髙田
24       西村         孫
25       柴田        一澤
26       池田        小山
27       久保        河合
28       竹田         孫
29        李        井上
30       猪原        柴田


## 6.2 Table of Calendar type of Duty days


In [173]:
calendar = make_calendar(table, year, month, start, end)


                    Mon  Tue  Wed  Thu  Fri  Sat  Sun
Week     Duty                                        
1st week day        NaN  NaN  NaN  NaN  NaN   *1   *2
         icu_duty   NaN  NaN  NaN  NaN  NaN   猪原   西村
         call_duty  NaN  NaN  NaN  NaN  NaN  清水と   一澤
2st week day          3    4    5    6    7   *8   *9
         icu_duty    上田    李    孫   池田   髙田   久保   寺西
         call_duty   荒井   寺西   西村  清水と   竹田    北   荒井
3st week day         10   11   12   13   14  *15  *16
         icu_duty    河合   西村   竹田   小山   河合   柴田   猪原
         call_duty    李   一澤  清水と   久保   池田    李   荒井
4st week day         17   18   19   20   21  *22  *23
         icu_duty    上田   寺西   髙田   池田    孫   猪原   小山
         call_duty   河合   一澤   竹田   小山   久保   寺西   髙田
5st week day         24   25   26   27   28  *29  *30
         icu_duty    西村   柴田   池田   久保   竹田    李   猪原
         call_duty    孫   一澤   小山   河合    孫   井上   柴田


## 6.3 Make a members info dataframe Adding Marked duty days of no_holidays

In [114]:
for name in members_info.values():
    # for n in name['icu_duty']:
    #     if n not in info_days[3]:
    #         name['no_holidays'] +=1
    # for n in name['call_duty']:
    #     if n in (info_days[0]+ info_days[1]):
    #         name['no_holidays'] +=1
    name['hope_icu_duty'] = [f'*{n}'  if n not in info_days[3] else n for n in name['hope_icu_duty']]
    name['hope_call_duty'] = [f'*{n}' if n not in info_days[3] else n for n in name['hope_call_duty']]
    name['icu_duty'] = [f'*{n}'  if n not in info_days[3] else n for n in name['icu_duty']]
    name['call_duty'] = [f'*{n}' if n not in info_days[3] else n for n in name['call_duty']]

In [115]:
df = pd.DataFrame(members_info)
df

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
no_icu_duty,True,True,True,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False
no_cardiac_surgery,True,False,False,False,True,True,False,False,False,True,False,True,False,False,False,False,False,False
no_additional_duty,True,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False
old_boy,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
unavailable_days,"[6, 7, 20, 21, 22, 23]","[5, 22, 23, 28]","[1, 2, 5, 15, 20, 21, 22]","[5, 6]","[1, 2, 5, 7, 14, 15, 16, 22, 28]","[1, 6, 7, 8, 12, 13, 14, 15, 16, 21, 28]","[1, 5, 7, 8, 9, 10, 12, 14, 15, 16, 22, 23, 26...","[6, 16]",[6],"[5, 7, 9, 21, 27]",[5],"[1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 17, 18, 24, 2...","[1, 2, 6, 8, 9, 10, 15, 16, 29]","[5, 7, 14, 15, 16, 29]","[1, 2, 6, 14, 15, 21, 22]","[1, 2, 3, 5, 7, 10, 17, 21, 22, 23, 24, 28]","[5, 7]","[5, 6, 7, 8, 20, 21, 22]"
hope_icu_duty,[],[],[],[],[],"[3, 17]",[],[],[],"[*1, *16, *22, *30]",[],[],[],[],[19],[26],[],[]
hope_call_duty,[*8],[],[],[],[*29],[],[*2],[],[],[],[],[],[],[],[],[],[],[]
icu_duty,[],[],[],"[*15, 24]",[],"[3, 17]",[],"[*8, 12]","[*21, 25]","[*1, *16, *22, *30]","[*14, 20]","[5, *28]","[18, *23]","[*9, 13]","[*7, 19]","[4, 11, 26]","[6, 10, *29]","[*2, 27]"
call_duty,[*8],"[*1, 4, 18]","[17, *23, 27]","[11, 19]",[*29],[],"[*2, *21, 25]","[5, 24, *28]",[*9],[],"[3, *7]","[*14, 20]","[12, *30]","[6, *22]","[13, *16]",[],[26],"[10, *15]"
duty_score,-106,-7,-7,-7,-106,-110,-7,-10,-7,-126,-7,-7,-9,-9,-10,-4,-9,-9


In [116]:
calendar

Mon  Tue  Wed  Thu  Fri  Sat  Sun
Week     Duty                                        
1st week day        NaN  NaN  NaN  NaN  NaN   *1   *2
         icu_duty   NaN  NaN  NaN  NaN  NaN   猪原    李
         call_duty  NaN  NaN  NaN  NaN  NaN  清水と   一澤
2st week day          3    4    5    6    7   *8   *9
         icu_duty    上田   池田   小山   久保   髙田   西村   寺西
         call_duty   河合  清水と   西村   寺西   河合    北    孫
3st week day         10   11   12   13   14  *15  *16
         icu_duty    久保   池田   西村   寺西   河合   柴田   猪原
         call_duty    李   柴田   竹田   髙田   小山    李   髙田
4st week day         17   18   19   20   21  *22  *23
         icu_duty    上田   竹田   髙田   河合    孫   猪原   竹田
         call_duty   荒井  清水と   柴田   小山   一澤   寺西   荒井
5st week day         24   25   26   27   28  *29  *30
         icu_duty    柴田    孫   池田    李   小山   久保   猪原
         call_duty   西村   一澤   久保   荒井   西村   井上   竹田

## 6.4 Make a excel file of above tables

In [117]:
with pd.ExcelWriter(f'//Users/taehunlee/Desktop/duty_schedule_ver2/output{year}_{month}.xlsx') as writer:
    calendar.to_excel(writer, sheet_name='Sheet1', index=True)
    table.to_excel(writer, sheet_name='Sheet2', index=True)
    df.to_excel(writer, sheet_name='Sheet3', index=True)